# Bitcoin derivation and transaction manual

In [1]:
from pybip44 import HDPrivateKey
import requests
import pandas as pd
from bitcoin_gp import BitcoinGP, Fee

## Direvation

In [2]:
master_key = HDPrivateKey.master_key_from_mnemonic(
    'cram flower anger behave science furnace match clip about rocket sell exist', 'btc')

# to switch between test and main network, you need to change 'coin' in derivation path m/44'/{coin}'/0
# for Main coin is 0
# for Test coin is 1
root_key = HDPrivateKey.from_path(master_key, "m/44'/0'/0'")

private_keys = []
addresses = []

for i in range(10):
    private_key = HDPrivateKey.from_path(root_key, '{change}/{index}'.format(change=0, index=i))
    private_keys.append(private_key.to_hex())
    public_key = private_key.public_key
#     print(f'\tPublic Key: {public_key.to_hex()}')
#     print(f'\tPrivate Key: {private_key.to_hex()}')
    
    # and here you need to change main_net parameter
    addresses.append(public_key.address(main_net=True))

    
df = pd.DataFrame(addresses, columns = ['Address']) 

df

,Address
0,146rDUFCLC3FVtDoj4eCv6pgy878e5eBVe
1,1LdktPmHUPER3ynuYf4jeDkb8iRdBgaW5f
2,1PbBz56AnXMfjvEi1HWNAcTy8Uh8EgRG9y
3,13CMG7btSZ4qputapZyXN2vbeggZaBRAkM
4,16NZ2PVzQsiNs3weks45uN7dMey8zdy3R1
5,1NHYDobZyewo9Vb45rwEnZgqf1FBxoc8yz
6,19MyH7uKv77Jb7FUGiRfgUykYkyiusojdo
7,1LjZiqkKCKTLTuqC7r5hi6H94Jy4wXFeay
8,17Jm4XrC2tT9tNeD6z9n8aah9swo1ZCEdW
9,13pVf73RMXrqzyLu1g6etCC26MXggLY9rK


## Balances

In [3]:
decimals = 10 ** 8

# here test net node on https://testgate.geniepay.io
# and main net on https://gate.geniepay.io
url = 'https://gate.geniepay.io'

balances = []

for address in addresses:
    response = requests.get(f'{url}/{address}/balance')
    if response:
        balances.append(response.json()['confirmed'] / decimals)

df = pd.DataFrame(zip(addresses, balances), columns = ['Address', 'Balance (confirmed)']) 

df

,Address,Balance (confirmed)
0,146rDUFCLC3FVtDoj4eCv6pgy878e5eBVe,0.005121
1,1LdktPmHUPER3ynuYf4jeDkb8iRdBgaW5f,0.000012
2,1PbBz56AnXMfjvEi1HWNAcTy8Uh8EgRG9y,0.000000
3,13CMG7btSZ4qputapZyXN2vbeggZaBRAkM,0.000000
4,16NZ2PVzQsiNs3weks45uN7dMey8zdy3R1,0.000000
5,1NHYDobZyewo9Vb45rwEnZgqf1FBxoc8yz,0.000000
6,19MyH7uKv77Jb7FUGiRfgUykYkyiusojdo,0.000000
7,1LjZiqkKCKTLTuqC7r5hi6H94Jy4wXFeay,0.000000
8,17Jm4XrC2tT9tNeD6z9n8aah9swo1ZCEdW,0.000000
9,13pVf73RMXrqzyLu1g6etCC26MXggLY9rK,0.000000


## Transaction

In [4]:
# https://testgate.geniepay.io  - testnet
# https://gate.geniepay.io  - mainnet

account_from = [addresses[0], private_keys[0]]
account_to = [addresses[1], private_keys[1]]

btc_gp = BitcoinGP(url, True)

btc_gp.send_transaction(private_key=account_from[1],
                     address_from=account_from[0],
                     address_to=account_to[0],
                     amount=0.00001,
                     fee=Fee.SLOW)


Transaction hash:
d2c8f1f656f056af56199328f17e71df2be4dd77fe26c06c0355e23e15fe91ed

Link to BlockCypher:
https://live.blockcypher.com/btc/tx/d2c8f1f656f056af56199328f17e71df2be4dd77fe26c06c0355e23e15fe91ed
